In [2]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import pandas as pd
from pandas import ExcelWriter
from collections import Counter
import math

print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(level=2)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(level=2)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(level=2)
print("Initializing drivers ... done")
print()


Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER
Initializing drivers ... done



In [9]:
# Input text
fileName="開發金"
filename = "./_target/"+fileName+".xls"
result="./_target/"+fileName+"reslt.xls"
file = pd.read_excel(filename)
print(len(file))
print(file.head(1))



8
         公開/公告號    公開/公告日          申請號       申請日  \
0  TW201915891A  20190416  TW106133599  20170929   

                                                專利名稱  \
0  利率遞減型還款系統Interest rate declining repayment system   

                                                申請人  \
0  凱基商業銀行股份有限公司; KGI COMMERCIAL BANK CO., LTD. (TW)   

                                                  摘要                  IPC  \
0  一種利率遞減型還款系統包含一個終端機及一個電腦主機，該電腦主機包括一個儲存單元及一個處理單元...  G06Q 40/02(2012.01)   

   引用專利  被參考次數  
0   NaN    NaN  


In [52]:
var=file["摘要"]
name=file["專利名稱"]
sort_key=[]
sort_words=[]
for i in range (3) :
   
    text =[var[i]]
    sent_cnt=0
    
    self_list=[]## get combine word 
    common_list=[]## get the most common word 
    document=[]
    #Run pipeline
    #print("Running pipeline ... WS")
    ws = ws_driver(text)
    #print("Running pipeline ... POS")
    pos = pos_driver(ws)
    #print("Running pipeline ... NER")
    
    ner = ner_driver(text)
    
    print("Running pipeline ... done")
    
    print()
    def pack_ws_pos_sentece(sentence_ws, sentence_pos):
        assert len(sentence_ws) == len(sentence_pos)
        res = []
             
        ## 字詞的組合 根據屬性表將名詞-> 專有名詞 來增加字詞的特殊性
        pre_pos=''
        pre_ws=''
        for word_ws, word_pos in zip(sentence_ws, sentence_pos):
            ## 寫的好爛... 可以透過語言結構做優化
            res.append(f"{word_ws}({word_pos})")
            
            if(word_pos=="Na" or "V" in word_pos):
                common_list.append(word_ws)
                
            if((word_pos == "Na"or word_pos=="VC") and pre_pos!='' ):
            # use 
                Nb=f"{pre_ws}{word_ws}"
                #print(Nb)
                self_list.append(pre_ws+word_ws)
                #res.append(f"{Nb}(Nb)")
                pre_pos=""
                pre_ws=""
                    
            else :
                ## memorizing
                if(word_pos=="VC" or word_pos=="VD" or word_pos=="Na" or word_pos=="Nb"):
                    pre_pos=word_pos
                    pre_ws=word_ws
                    
                else :
                    pre_pos=''
                    pre_ws=''
                    #res.append(f"{word_ws}({word_pos})")
            if(word_pos=="PERIODCATEGORY" or word_pos=="SEMICOLONCATEGORY"):
                global sent_cnt,document
                sent_cnt+=1
                #print(word_ws+""+str(sent_cnt))
                
                document.append(res)
            
        return "\u3000".join(res)
    
    
    ## zip指標的標記
    for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
        #print(pos)
        pack_ws_pos_sentece(sentence_ws, sentence_pos)
        #-----------------------------------------------#
    sort_key.append(Counter(self_list).most_common(20))
    sort_words.append(Counter(common_list).most_common(20))
    print(name[i])
    print("done")
    #print(sentence)
    print("-----weighting-------")
#     print(sort_key[i])
#     print("------------")
#     print(sort_words[i])
    weight=[]
    for words in sort_words[i]:
        _TF=(words[1]/len(pos)) 
        _IDF=math.log(words[1]/sent_cnt)
        print(words[0])
        print(abs(_TF*_IDF))
        print()
        weight=zip(words[0],abs(_TF*_IDF))
    
    # for entity in sentence_ner:
    print(weight)
    print("----storing------")
    del text
    del self_list,common_list,document 


Inference: 100%|██████████| 1/1 [00:03<00:00,  3.42s/it]

Running pipeline ... done

利率遞減型還款系統Interest rate declining repayment system
done
-----weighting-------
還款
2.4327906486489863



TypeError: 'float' object is not iterable

In [49]:
## export back to the file 

print(weight)


{'系統': 2.4327906486489863, '交易': 1.1157177565710488, '控制': 1.1157177565710488, '係': 1.1157177565710488, '模組': 1.1157177565710488, '金融': 0.8630462173553427, '商品': 0.8630462173553427, '策略': 0.8630462173553427, '行動': 0.8630462173553427, '裝置': 0.8630462173553427, '電訊': 0.8630462173553427, '連接': 0.8630462173553427, '通訊': 0.8630462173553427, '查看': 1.3862943611198906, '電腦': 1.3862943611198906, '主機': 1.3862943611198906, '供以': 1.3862943611198906, '運行': 1.3862943611198906, '有': 1.3862943611198906, '創作': 1.3862943611198906}


In [51]:
bank ={
        "KEY":sort_key,
        "WORD":weight,
        
                }
print (pd.DataFrame(bank))

pd.DataFrame(bank).to_excel(result)



ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.